# Lab #1

---

## Task
Для СЛАУ с некоторой матрицей A:

- вычислить числа обусловленности
- поварьировав матрицу и правую часть, вычислить |x - ͂x͂|
- посмотреть, есть ли корреляция между величинами чисел обусловленности и погрешностью решения

## Solution
---

### Computation of condition numbers

In [ ]:
from scipy import linalg as la
import numpy as np
import math

def spectral_condition_nums(matr):
    return la.norm(matr) * la.norm(la.inv(matr))

def bulk_condition_nums(matr):
    product = 1
    n, m = matr.shape
    
    for i in range(n):
        temp_sum = 0
        for j in range(m):
            temp_sum += pow(matr[i, j], 2)
        product *= math.sqrt(temp_sum)
    
    return product / abs(la.det(matr))

def angular_condition_nums(matr):
    iterator = zip(matr, la.inv(matr).transpose())
    return max([la.norm(row) * la.norm(col) for (row, col) in iterator])

def compute_matr_condition_nums(matr):
    return [
        spectral_condition_nums(matr),
        bulk_condition_nums(matr),
        angular_condition_nums(matr)
    ]

### Linear equation solution

In [ ]:
def solve_le(A, b):
    return la.solve(A, b)

### Creation of test matrices

In [ ]:
def round_matrices(matrices, decimals):
    return [matrix.round(decimals) for matrix in matrices]

def create_hilbert_matrices(range):
    return [la.hilbert(i) for i in range]

def create_tridiagonal_matrix(size):
    if size > 2:
        zeros = [0 for _ in range(size - 2)]
    else:
        raise AttributeError('size should be greater than 2.')
    return np.array(la.toeplitz([2, -1] + zeros, [0, -1] + zeros))

def create_tridiagonal_matrices(range):
    return [create_tridiagonal_matrix(i) for i in range]

def create_random_matrices(range):
    return [np.random.random((i, i)) for i in range]

## Experimental research

---

This block presents results of the implemented functions.
We evaluate three different test cases:
- Hilbert matrices
- Randomly generated matrices
- Diagonally dominant tridiagonal matrices
    
For each case we vary matrices and vectors of linear system to check the correlation between
condition numbers and solution error.

In [ ]:
import pandas as pd

ranks = range(3, 20)
decimals = range(2, 11, 3)

hilbert_matrices = create_hilbert_matrices(ranks)
tridiagonal_matrices = create_tridiagonal_matrices(ranks)
random_matrices = create_random_matrices(ranks)

def experiment(matr, decimal_round):
    n, m = matr.shape
    id_vect = np.ones(m)
    
    b = matr.dot(id_vect)
    b = b.round(decimal_round)
    matr = matr.round(decimal_round)
    
    sol = solve_le(matr, b)
    
    return la.norm(sol - id_vect)

### Results for Hilbert matrices

In [ ]:
res_hilbert = []
cond_nums = []

for matr in hilbert_matrices:
    hilbert_results = [experiment(matr, decimal) for decimal in decimals]
    res_hilbert.append(hilbert_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df_h = pd.DataFrame(res_hilbert, index=ranks, columns=decimals)
df_hcn = pd.DataFrame(cond_nums, index=ranks, columns=["spectral", "bulk", "angular"])

df_h = df_h.round(decimals=6)
df_hcn = df_hcn.round(decimals=3)

print(pd.concat([df_h, df_hcn], axis=1))

### Results for randomly generated matrices

In [ ]:
res_random = []
cond_nums = []

for matr in random_matrices:
    random_matrices_results = [experiment(matr, decimal) for decimal in decimals]
    res_random.append(random_matrices_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df_r = pd.DataFrame(res_random, index=ranks, columns=decimals)
df_rcn = pd.DataFrame(cond_nums, index=ranks, columns=["spectral", "bulk", "angular"])

df_r = df_r.round(decimals=6)
df_rcn = df_rcn.round(decimals=3)

print(pd.concat([df_r, df_rcn], axis=1))

### Result for diagonally dominant tridiagonal matrices

In [ ]:
res_tridiag = []
cond_nums = []

for matr in tridiagonal_matrices:
    tridiag_results = [experiment(matr, decimal) for decimal in decimals]
    res_tridiag.append(tridiag_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df = pd.DataFrame(res_tridiag, index=ranks, columns=decimals)
df1 = pd.DataFrame(cond_nums, index=ranks, columns=["spectral", "bulk", "angular"])

df = df.round(decimals=9)
df1 = df1.round(decimals=3)

print(pd.concat([df, df1], axis=1))